# Tankage

## Overview

Copyright 2023 Moran Innovation LLC

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.

CryoFM™ is a trademark of Moran Innovation LLC. All rights reserved.

### Usage Notes

**HOW TO USE THIS INTERACTIVE NOTEBOOK:**
* Use the table of contents icon and sidebar at the left to navigate to the topics of interest.
* Change any of the input values and run the associated cell code.
* If an interactive Binder web browser session is being used, the original file will not be altered. Therefore, if you wish to retain any modifications you make, save your file to local memory on your device.

**CAUTION:** None of the code cells run automatically. The user must manually run selected code cells in sequence to achieve the desired results. Default units are SI.

**REFERENCE REPORT AND TRAINING:** The reference report for this interactive notebook is "Cryogenic fluid management of liquid hydrogen, oxygen, and methane: Part 1 - passive technologies, systems, and operations",  Moran Innovation LLC, 2023. An online version of the report and training videos based on the content can be found here: https://sites.google.com/view/matthewemoran/training#h.ir8xm9d8wn6h.

## Material Properties

Many material properties are a nonlinear as a function of temperature in the cryogenic range.
Mechanical properties that can vary significantly at low temperature include [Flynn, Cryogenic Engineering, 2nd ed., 2005]:
* Yield and ultimate strength: generally increases at lower temperatures for most solids
* Ductility: some materials remain ductile (e.g., aluminum alloys, austenitic stainless steel
with > 7% nickel, most face-centered cubic metals); while some materials become brittle
(carbon steel, most plastics, most body-centered cubic metals)
* Elastic modulus: varies
* Fatigue strength: varies

Thermal properties are highly temperature dependent at cryogenic temperatures including
thermal conductivity and specific heat. Accurate analysis of conduction heat transfer and thermal
energy capacitance require integration of these properties over the relevant temperature range.

NIST provides an online calculator for this purpose at: https://trc.nist.gov/cryogenics/calculators/graphcalc.html. Curve fits of data or published integrals for selected materials can also be used.

### Solid Thermal Conduction

In [8]:
# Solid conduction heat transfer
description = 'stainless steel 304 pipe: 110 K to 300 K'
shape_factor = 0.208  # m, crossectional area divided by length (or equiv for other shapes)
cond_integ = 2433  # W/m, from conduction integral table or NIST online calculator for temp range
cond_ht = shape_factor * cond_integ

print(description)
print("Conduction heat transfer =", "{:.1f}".format(cond_ht), "W")

stainless steel 304 pipe: 110 K to 300 K
Conduction heat transfer = 506.1 W


### Solid Thermal Capacitance (heat absorbed or dissipated)

In [9]:
# Solid heat absorbed or dissipated (equation 3.2)
description = 'stainless steel 304 structure: chilldown from 300 K to 110 K'
mass = 1.  # kg, material mass
cp_integ = 78136 # J/kg-K, from NIST online calculator using initial and final temps
heat_out = mass * cp_integ/1000  # kJ, heat in (+) or out(-)

print(description)
print("Heat absorbed or dissipated =", "{:.1f}".format(heat_out), "kJ")

stainless steel 304 structure: chilldown from 300 K to 110 K
Heat absorbed or dissipated = 78.1 kJ


## Calculation Examples